In [1]:
import pandas as pd
import os
from os.path import dirname

root_path = dirname(os.getcwd())

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI_Challenge_2012_W_Complete"
data_dir_dataset_encoders = root_path + "/data/datasets/encoders/" + dataset
filename = data_dir + dataset + ".csv"
raw_data = pd.read_csv(filename, index_col=False)
raw_data = raw_data.drop(columns="Unnamed: 0")

In [3]:
tab_all = raw_data.rename(
    columns={"case:concept:name": "CaseID", "concept:name": "Activity"}
)
# tab_all.head()

In [4]:
tab_all["time:timestamp"] = [x.split(".")[0] for x in tab_all["time:timestamp"]]
tab_all["case:REG_DATE"] = [x.split(".")[0] for x in tab_all["case:REG_DATE"]]
tab_all["time:timestamp"] = tab_all["time:timestamp"].str.replace("-", "/")
tab_all["time:timestamp"] = tab_all["time:timestamp"].str.split("+", expand=True)[0]
tab_all["case:REG_DATE"] = tab_all["case:REG_DATE"].str.replace("-", "/")
tab_all["case:REG_DATE"] = tab_all["case:REG_DATE"].str.split("+", expand=True)[0]
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,2011/10/01 10:10:25,2011/10/01 09:58:30,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,2011/10/01 13:03:35,2011/10/01 09:58:30,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,2011/10/03 11:17:29,2011/10/01 09:58:30,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,2011/10/03 14:42:55,2011/10/01 09:58:30,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,2011/10/01 10:16:49,2011/10/01 09:45:37,173706,18000


In [5]:
from utils import translate_time

tab_all['time:timestamp'] = tab_all['time:timestamp'].apply(translate_time)
tab_all['case:REG_DATE'] = tab_all['case:REG_DATE'].apply(translate_time)
tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,1.317457e+09,1.317456e+09,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,1.317467e+09,1.317456e+09,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,1.317633e+09,1.317456e+09,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,1.317646e+09,1.317456e+09,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,1.317457e+09,1.317455e+09,173706,18000


In [ ]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

In [6]:
from math import log
import numpy as np

min_time = tab_all['time:timestamp'].min() if tab_all['time:timestamp'].min() < tab_all['case:REG_DATE'].min() else tab_all['case:REG_DATE'].min()

tab_all['time:timestamp'] -= min_time
tab_all['time:timestamp'] = [ log(x)  if x > 0 else 0. for x in tab_all['time:timestamp'].values]
# tab_all['time:timestamp'] = np.log(tab_all['time:timestamp'])

tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,10.442930,1.317456e+09,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,10.707527,1.317456e+09,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,12.260206,1.317456e+09,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,12.316947,1.317456e+09,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,10.454063,1.317455e+09,173706,18000


In [7]:
tab_all['case:REG_DATE'] -= min_time
tab_all['case:REG_DATE'] = [ log(x)  if x > 0 else 0. for x in tab_all['case:REG_DATE'].values]
# tab_all['time:timestamp'] = np.log(tab_all['time:timestamp'])

tab_all.head()

,org:resource,lifecycle:transition,Activity,time:timestamp,case:REG_DATE,CaseID,case:AMOUNT_REQ
0,10912.0,COMPLETE,W_Afhandelen leads,10.442930,10.421865,173712,30000
1,11019.0,COMPLETE,W_Completeren aanvraag,10.707527,10.421865,173712,30000
2,11180.0,COMPLETE,W_Completeren aanvraag,12.260206,10.421865,173712,30000
3,11180.0,COMPLETE,W_Completeren aanvraag,12.316947,10.421865,173712,30000
4,10912.0,COMPLETE,W_Afhandelen leads,10.454063,10.398580,173706,18000


In [8]:
tab_all['case:AMOUNT_REQ'] = [log(int(x)) if x > 0 else 0 for x in tab_all['case:AMOUNT_REQ'].values]

In [ ]:
tab_all.head()

# Discover roles

In [ ]:
from scipy.stats import pearsonr
import networkx as nx

from operator import itemgetter
import pandas as pd


class ResourcePoolAnalyser():
    """
        This class evaluates the tasks durations and associates resources to it
     """

    def __init__(self, log, drawing=False, sim_threshold=0.7):
        """constructor"""
        self.data = self.read_resource_pool(log)
        self.drawing = drawing
        self.sim_threshold = sim_threshold
        
        self.tasks = {val: i for i, val in enumerate(self.data["Activity"].unique())}
        self.users = {val: i for i, val in enumerate(self.data["org:resource"].unique())}
        
        self.roles, self.resource_table = self.discover_roles()

    def read_resource_pool(self, log):
        if isinstance(log, pd.DataFrame):
            filtered_list = log[['Activity', 'org:resource']]
        else:
            filtered_list = pd.DataFrame(log.data)[['Activity', 'org:resource']]
        #filtered_list = filtered_list[~filtered_list.task.isin(['Start', 'End'])]
        #filtered_list = filtered_list[filtered_list.user != 'AUTO']
        return filtered_list


    def discover_roles(self):
        associations = lambda x: (self.tasks[x['Activity']], self.users[x['org:resource']])
        self.data['ac_rl'] = self.data.apply(associations, axis=1)
    
        freq_matrix = (self.data.groupby(by='ac_rl')['Activity']
                       .count()
                       .reset_index()
                       .rename(columns={'Activity': 'freq'}))
        freq_matrix = {x['ac_rl']: x['freq'] for x in freq_matrix.to_dict('records')}
        
        profiles = self.build_profile(freq_matrix)
    
        print(((20 / 100)* 100),'Analysing resource pool ')
        # building of a correl matrix between resouces profiles
        correl_matrix = self.det_correl_matrix(profiles)
        print(((40 / 100)* 100),'Analysing resource pool ')
        # creation of a rel network between resouces
        g = nx.Graph()
        for user in self.users.values():
            g.add_node(user)
        for rel in correl_matrix:
            # creation of edges between nodes excluding the same elements
            # and those below the similarity threshold 
            if rel['distance'] > self.sim_threshold and rel['x'] != rel['y']:
                g.add_edge(rel['x'],
                           rel['y'],
                           weight=rel['distance'])
        print(((60 / 100) * 100),'Analysing resource pool ')
        # extraction of fully conected subgraphs as roles
        sub_graphs = list(nx.connected_components(g))
        print(((80 / 100) * 100),'Analysing resource pool ')
        # role definition from graph
        roles = self.role_definition(sub_graphs)
        # plot creation (optional)
        # if drawing == True:
        #     graph_network(g, sub_graphs)
        print(((100 / 100)* 100),'Analysing resource pool ')
        
        
        import pprint
        pprint.pprint(f"ROLES \n\n{roles}\n\n")
        return roles
    
    def build_profile(self, freq_matrix):
        profiles=list()
        for user, idx in self.users.items():
            profile = [0,] * len(self.tasks)
            for ac_rl, freq in freq_matrix.items():
                if idx == ac_rl[1]:
                    profile[ac_rl[0]] = freq
            profiles.append({'user': idx, 'profile': profile})
        return profiles



    def det_correl_matrix(self, profiles):
        correl_matrix = list()
        import numpy
        for profile_x in profiles:
            for profile_y in profiles:
                x = numpy.array(profile_x['profile'])
                y = numpy.array(profile_y['profile'])
                r_row, p_value = pearsonr(x, y)
                correl_matrix.append(({'x': profile_x['user'],
                                            'y': profile_y['user'],
                                            'distance': r_row}))
        return correl_matrix

    def role_definition(self, sub_graphs):
        user_index = {v: k for k, v in self.users.items()}
        records= list()
        for i in range(0, len(sub_graphs)):
            users_names = [user_index[x] for x in sub_graphs[i]]
            records.append({'role': 'Role '+ str(i + 1),
                            'quantity': len(sub_graphs[i]),
                            'members': users_names})
        #Sort roles by number of resources
        records = sorted(records, key=itemgetter('quantity'), reverse=True)
        for i in range(0,len(records)):
            records[i]['role']='Role '+ str(i + 1)
        resource_table = list()
        for record in records:
            for member in record['members']:
                resource_table.append({'role': record['role'],
                                       'resource': member})
        return records, resource_table
    
    
def get_resource_role_map(log):
    r = ResourcePoolAnalyser(log)
    return r.resource_table

In [9]:
tab_all = tab_all.fillna({key: "NAN" for key in ["org:resource"]})
# tab_all = tab_all.fillna({key: -1 for key in real_value_columns})

In [10]:
from utils import get_resource_role_map
roles_map = get_resource_role_map(tab_all)

/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/utils.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['ac_rl'] = self.data.apply(associations, axis=1)


20.0 Analysing resource pool 
40.0 Analysing resource pool 
60.0 Analysing resource pool 
80.0 Analysing resource pool 
100.0 Analysing resource pool 
('ROLES \n'
 '\n'
 "([{'role': 'Role 1', 'quantity': 49, 'members': [10912.0, 11019.0, 11180.0, "
 "'NAN', 10982.0, 11002.0, 11049.0, 11122.0, 10913.0, 10889.0, 11121.0, "
 '10939.0, 11009.0, 11201.0, 11119.0, 10861.0, 11203.0, 11181.0, 11189.0, '
 '10899.0, 11000.0, 10863.0, 11169.0, 11179.0, 11001.0, 10228.0, 10909.0, '
 '10789.0, 10881.0, 10910.0, 10929.0, 10931.0, 11259.0, 10779.0, 10914.0, '
 '10933.0, 11079.0, 10932.0, 10935.0, 11254.0, 11003.0, 11269.0, 112.0, '
 "11299.0, 10124.0, 11309.0, 11300.0, 11302.0, 11319.0]}, {'role': 'Role 2', "
 "'quantity': 9, 'members': [10629.0, 11339.0, 10972.0, 10609.0, 10821.0, "
 "11289.0, 10125.0, 10809.0, 10138.0]}, {'role': 'Role 3', 'quantity': 2, "
 "'members': [11304.0, 10188.0]}], [{'role': 'Role 1', 'resource': 10912.0}, "
 "{'role': 'Role 1', 'resource': 11019.0}, {'role': 'Role 1', 're

In [11]:
roles_map = {
    x["resource"] : x["role"]
    for x in roles_map
}
roles_map

{10912.0: 'Role 1',
 11019.0: 'Role 1',
 11180.0: 'Role 1',
 'NAN': 'Role 1',
 10982.0: 'Role 1',
 11002.0: 'Role 1',
 11049.0: 'Role 1',
 11122.0: 'Role 1',
 10913.0: 'Role 1',
 10889.0: 'Role 1',
 11121.0: 'Role 1',
 10939.0: 'Role 1',
 11009.0: 'Role 1',
 11201.0: 'Role 1',
 11119.0: 'Role 1',
 10861.0: 'Role 1',
 11203.0: 'Role 1',
 11181.0: 'Role 1',
 11189.0: 'Role 1',
 10899.0: 'Role 1',
 11000.0: 'Role 1',
 10863.0: 'Role 1',
 11169.0: 'Role 1',
 11179.0: 'Role 1',
 11001.0: 'Role 1',
 10228.0: 'Role 1',
 10909.0: 'Role 1',
 10789.0: 'Role 1',
 10881.0: 'Role 1',
 10910.0: 'Role 1',
 10929.0: 'Role 1',
 10931.0: 'Role 1',
 11259.0: 'Role 1',
 10779.0: 'Role 1',
 10914.0: 'Role 1',
 10933.0: 'Role 1',
 11079.0: 'Role 1',
 10932.0: 'Role 1',
 10935.0: 'Role 1',
 11254.0: 'Role 1',
 11003.0: 'Role 1',
 11269.0: 'Role 1',
 112.0: 'Role 1',
 11299.0: 'Role 1',
 10124.0: 'Role 1',
 11309.0: 'Role 1',
 11300.0: 'Role 1',
 11302.0: 'Role 1',
 11319.0: 'Role 1',
 10629.0: 'Role 2',
 113

In [12]:
tab_all["org:resource:role"] = [roles_map[role] for role in tab_all["org:resource"].values]

In [ ]:
tab_all

In [13]:
tab_all["org:resource"] = [str(x) for x in tab_all["org:resource"].values]

In [14]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

# Split the dataset

In [ ]:
split_ratio = 2 / 3

first_act_tab = (
    tab_all.groupby("CaseID").first().sort_values("time:timestamp").reset_index()
)
first_act_tab = first_act_tab[
    ~first_act_tab.duplicated(subset=["CaseID", "Activity"], keep="first")
]
first_act_tab = first_act_tab.reset_index(drop=True)

list_train_valid_cases = list(
    first_act_tab[: int(split_ratio * len(first_act_tab))]["CaseID"].unique()
)

list_train_cases = list_train_valid_cases[: int(len(list_train_valid_cases) * 0.8)]
tab_train = tab_all[tab_all["CaseID"].isin(list_train_cases)].reset_index(drop=True)
# tab_train.to_csv(data_dir_processed+ f"{dataset}_processed_train.csv", index = False)

list_valid_cases = list_train_valid_cases[int(len(list_train_valid_cases) * 0.8) :]
tab_valid = tab_all[tab_all["CaseID"].isin(list_valid_cases)].reset_index(drop=True)
# tab_valid.to_csv(data_dir_processed+f"{dataset}_processed_valid.csv", index = False)

list_test_cases = list(
    first_act_tab[int(split_ratio * len(first_act_tab)) :]["CaseID"].unique()
)
tab_test = tab_all[tab_all["CaseID"].isin(list_test_cases)].reset_index(drop=True)
# tab_test.to_csv(data_dir_processed+ f"{dataset}_processed_test.csv", index = False)

In [ ]:
tab_all['org:resource'].value_counts()

In [ ]:
list(tab_all['Activity'].values)

## Encoders for categorical attributes


In [ ]:
from torch import nn

class AutoEncoder(nn.Module):
    def __init__(self, n_categories):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(n_categories, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.Sigmoid()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, n_categories),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
cat_categories = ["Activity", "org:resource"]

In [ ]:
import torch
import os.path as path

encoders = {k: AutoEncoder(n_categories= len(tab_all[k].unique())) for k in cat_categories}
print()
for k in encoders:
    encoders[k].load_state_dict(torch.load(path.join(data_dir_dataset_encoders, f"encoder_{k}.pt"), weights_only=True))


### Prepare the graphs

In [ ]:
from utils import get_case_ids, is_static, get_one_hot_encoder, get_one_hot_encodings
import utils
from torch import tensor, max, int64, float32
from torch_geometric.data import HeteroData
import numpy as np

In [ ]:
def get_node_features(dataset: pd.DataFrame, trace: pd.DataFrame) -> dict:
    columns_static = [c for c in trace if is_static(trace[c])]

    res = {}

    for key in trace:
        values = trace[key].values
        match key:
            case "Activity":
                onehot_activities = get_one_hot_encoder(dataset, "Activity")
                res[key] = tensor(
                    get_one_hot_encodings(onehot_activities, values), dtype=float32,requires_grad=True
                )
            case "time:timestamp":
                res[key] = tensor(
                    values,  dtype=float32,requires_grad=True
                )
                res[key] = res[key].reshape(res[key].shape[0], 1)
            case "org:resource":
                onehot_resource = get_one_hot_encoder(dataset, "org:resource")
                # if key not in columns_static:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_resource, values),
                    dtype=float32,
                    requires_grad=True
                )
            case "org:resource:role":
                onehot_resource_role = get_one_hot_encoder(dataset, "org:resource:role")
                # if key not in columns_static:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_resource_role, values),
                    dtype=float32,
                    requires_grad=True
                )               

            case "lifecycle:transition":
                onehot_lifecyle_transition = get_one_hot_encoder(
                    dataset, "lifecycle:transition"
                )
                if key not in columns_static:
                    res[key] = tensor(
                        get_one_hot_encodings(onehot_lifecyle_transition, values),
                        dtype=float32,
                        requires_grad=True
                    )
                else:
                    res[key] = tensor(
                        get_one_hot_encodings(
                            onehot_lifecyle_transition, np.array([values[0]])
                        ),
                        dtype=float32,
                        requires_grad=True
                    )
            case "case:REG_DATE":
                if key not in columns_static:
                    res[key] = tensor(
                        values, dtype=float32,requires_grad=True
                    )
                else:
                    res[key] = tensor(
                        [values[0]],
                        dtype=float32,
                        requires_grad=True
                    )
                res[key] = res[key].reshape(res[key].shape[0], 1)
            case "case:AMOUNT_REQ":
                if key not in columns_static:
                    res[key] = tensor(values, dtype=float32,requires_grad=True)
                else:
                    res[key] = tensor([values[0]], dtype=float32,requires_grad=True)
                res[key] = res[key].reshape(res[key].shape[0], 1)

    return res


In [ ]:


def compute_edges_indexs(node_features: dict, prefix_len):
    res = {}
    keys = node_features.keys()
    # indexes = [[i, j] for i in range(prefix_len) for j in range(i + 1, prefix_len)]
    indexes = [[i, i + 1] for i in range(prefix_len-1)]
    # activities indexes
    for k in keys:
        if len(node_features[k]) != 1:
            if k == "Activity":
                res[(k, "followed_by", k)] = indexes
                for k2 in keys:
                    if k2 != k:
                        if len(node_features[k2]) == 1:
                            res[(k, "related_to", k2)] = [
                                [i, 0] for i in range(prefix_len)
                            ]
                        else:
                            res[(k, "related_to", k2)] = [
                                [i, i] for i in range(prefix_len)
                            ]
            else:
                res[(k, "related_to", k)] = indexes

    return res

In [ ]:
from torch import cat


def compute_edges_features(node_features, edges_indexes):
    res = {}

    for k in edges_indexes:
        if k[0] == k[2]:
            indexes = edges_indexes[k]
            res[k] = []
            match k[0]:
                case "Activity":
                    for i in indexes:
                        res[k].append(
                            # cat( (tensor([cosine_similarity(node_features[k[0]][i[1]],node_features[k[0]][i[0]], dim=0)]),tensor([i[1] - i[0]])) )
                            cat(
                                (
                                    #node_features[k[0]][i[1]]
                                    #- node_features[k[0]][i[0]],
                                    #tensor([torch.cosine_similarity(node_features[k[0]][i[0]], node_features[k[0]][i[1]], dim=0)]),
                                    tensor([torch.equal(node_features[k[0]][i[0]],node_features[k[0]][i[1]])], dtype=torch.float32),
                                    tensor([i[1] - i[0]], dtype=torch.float32),
                                )
                            )
                        )
                case "org:resource":
                    for i in indexes:
                        res[k].append(
                            # cat( (tensor([cosine_similarity(node_features[k[0]][i[1]], node_features[k[0]][i[0]], dim=0)]),tensor([i[1] - i[0]])) )
                            cat(
                                (
                                    # node_features[k[0]][i[1]]
                                    # - node_features[k[0]][i[0]],
                                   #tensor([torch.cosine_similarity(node_features[k[0]][i[0]], node_features[k[0]][i[1]], dim=0)]),
                                   tensor([torch.equal(node_features[k[0]][i[0]],node_features[k[0]][i[1]])], dtype=torch.float32),
                                   tensor([i[1] - i[0]], dtype=torch.float32),
                                )
                            )
                        )
                case "org:resource:role":
                    for i in indexes:
                        res[k].append(
                            # cat( (tensor([cosine_similarity(node_features[k[0]][i[1]], node_features[k[0]][i[0]], dim=0)]),tensor([i[1] - i[0]])) )
                            cat(
                                (
                                    # node_features[k[0]][i[1]]
                                    # - node_features[k[0]][i[0]],
                                   #tensor([torch.cosine_similarity(node_features[k[0]][i[0]], node_features[k[0]][i[1]], dim=0)]),
                                   tensor([torch.equal(node_features[k[0]][i[0]],node_features[k[0]][i[1]])], dtype=torch.float32),
                                   tensor([i[1] - i[0]], dtype=torch.float32),
                                )
                            )
                        )
                case "time:timestamp":
                    for i in indexes:
                        res[k].append(
                            tensor(
                                [
                                    node_features[k[0]][i[1]]
                                    - node_features[k[0]][i[0]],
                                    i[1] - i[0],
                                ]
                            )
                        )

    return res

In [ ]:
import torch

In [ ]:
# from pprint import pprint as print

from torch import stack
from math import log

def build_prefixes_graph_from_trace(dataset, trace):
    X = []  # graphs
    Y = []  # NA, timestamp, resource labels

    node_features = get_node_features(dataset, trace)
    
    #new_node_features = {}
    #for k in node_features:
    #    if k in cat_categories:
    #        new_node_features[k] = []
    #        for i in range(len(node_features[k])):
    #            new_node_features[k].append(encoders[k].encoder(node_features[k][i]))
    #        new_node_features[k] = torch.stack(new_node_features[k])
    #    else:
    #        new_node_features[k] = node_features[k]
    
    # !!!!!!!!!!!!!!!!!
    new_node_features = node_features
    
    prefix_lenghts = range(2, len(trace))
    # print(prefix_lenghts)
    for prefix in prefix_lenghts:
        # print(prefix)

        # init node types and features
        G = HeteroData()
        for k in new_node_features:
            if k == "org:resource":
                G[k].x = new_node_features[k][:prefix]
            else:
                G[k].x = new_node_features[k][:prefix]

        edges_indexes = compute_edges_indexs(new_node_features, prefix)

        edge_features = compute_edges_features(new_node_features, edges_indexes)
    
        for k in edge_features:
            if k[0] in ["Activity", "org:resource", "time:timestamp", "org:resource:role"]:
                G[k].edge_attr = stack(edge_features[k])
            else:
                G[k].edge_attr = tensor(edge_features[k], dtype=float32)

        for k in edges_indexes:
            ce = [[], []]
            for i in range(len(edges_indexes[k])):
                ce[0].append(edges_indexes[k][i][0])
                ce[1].append(edges_indexes[k][i][1])
            edges_indexes[k] = ce

        for k in edges_indexes:
            G[k].edge_index = tensor(edges_indexes[k], dtype=int64)

        
        G.y = {
            "Activity" : torch.max(node_features["Activity"][prefix],0)[1],
            "time:timestamp" :  torch.tensor([node_features["time:timestamp"][prefix][0]]),
            "org:resource" : torch.max(node_features["org:resource"][0],0)[1] if len(node_features["org:resource"]) == 1 else torch.max(node_features["org:resource"][prefix], 0)[1],
            "org:resource:role" : torch.max(node_features["org:resource:role"][0],0)[1] if len(node_features["org:resource:role"]) == 1 else torch.max(node_features["org:resource:role"][prefix], 0)[1],
        }
        
        X.append(G)

        
    return X

## Create the datasets

In [ ]:
case_train_ids = get_case_ids(tab_train)
case_valid_ids = get_case_ids(tab_valid)
case_test_ids = get_case_ids(tab_test)

In [ ]:
print(len(case_train_ids))
print(len(case_valid_ids))
print(len(case_test_ids))

In [ ]:
trace = (
        tab_train.query(f"CaseID == {case_train_ids[0]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
trace 

In [ ]:
graphs = build_prefixes_graph_from_trace(tab_all, trace)

In [ ]:
graphs[2].edge_attrs_dict

In [ ]:
import networkx as nx
import torch_geometric
import torch_geometric.utils
#g = torch_geometric.utils.to_networkx(graphs[0])
#nx.draw(g, with_labels=True,node_size=500, 
#    node_color='skyblue', 
#    font_weight='bold')

nx_graph = torch_geometric.utils.convert.to_networkx(graphs[0])

nx.draw_shell(nx_graph, with_labels=True)


In [ ]:
graphs[1].to_homogeneous().node_type

In [ ]:
graphs[1].x_dict

In [ ]:
graphs[1].y

In [ ]:
graphs[1].edge_items()

In [ ]:
from tqdm.notebook import tqdm


print("Preparing training dataset...")

X_train = []


for i in tqdm(range(len(case_train_ids))):
    trace = (
        tab_train.query(f"CaseID == {case_train_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace)
    
    # print(trace)
    # print([x.x_dict for x in graphs[:2]])
    # print(labels[:2])
    # break

    for i in range(len(graphs)):
        X_train.append(graphs[i])
        

# Y_train = tensor(Y_train, dtype=float32)

print("Done!\n\n")

In [ ]:
print("Preparing validation dataset...")

X_valid = []


for i in tqdm(range(len(case_valid_ids))):
    trace = (
        tab_valid.query(f"CaseID == {case_valid_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace)
    for i in range(len(graphs)):
        X_valid.append(graphs[i])
       

# Y_valid = tensor(Y_valid)

print("Done!\n\n")

In [ ]:
print("Preparing test dataset...")

X_test = []


for i in tqdm(range(len(case_test_ids))):
    trace = (
        tab_test.query(f"CaseID == {case_test_ids[i]}")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace)
    for i in range(len(graphs)):
        X_test.append(graphs[i])
     

# Y_test = tensor(Y_test)

print("Done!\n\n")

### Let's Save the graph datasets

In [ ]:
import pickle

with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_BOOOH.pkl", "wb") as f:
    pickle.dump(X_train, f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_BOOOH.pkl", "wb") as f:
    pickle.dump(X_valid, f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_BOOOH.pkl", "wb") as f:
    pickle.dump(X_test, f)